Say I've got a system of fairly simple relationships between variables: a = b + c, d = a*c, f = sqrt(a*b). Note there are implicit relations like c = a- b = f**2/b - b as well. Are there any ready to use libraries or modules that implement a constraint satisfaction network like this? I'd like the system start out without values assigned to any variables, but then the user inputs something like f = 3. This constrains the system now. If the user adds more values, at some point the system will be satisfied. As values are updated by the user, the system continue to ensure satisfaction, updating downstream values as necessary. I know this is generally a complex thing. I'd consider python-constraint but that is for finite-domains, and I'd like the system to support real number interval domains. Anyone know of some other similar projects out there? 

In [1]:
import sys   
sys.path.append(r'/home/erik/GitRepos/PowerElectronics')
for i in sorted(sys.path):
    print(i)


/home/erik/GitRepos/PowerElectronics
/home/erik/GitRepos/PowerElectronics/Chapter2Problems
/home/erik/anaconda3/envs/py312/lib/python3.12
/home/erik/anaconda3/envs/py312/lib/python3.12/lib-dynload
/home/erik/anaconda3/envs/py312/lib/python3.12/site-packages
/home/erik/anaconda3/envs/py312/lib/python312.zip


In [2]:
from IPython.display import display, Latex
from functools import wraps
from ordered_set import OrderedSet
import sympy
from sympy.core.symbol import symbols, Symbol
from sympy.solvers.solveset import nonlinsolve
from sympy import solveset
from sympy import Eq
from sympy.utilities.lambdify import lambdify
from sympy.printing import latex, pprint
from sympy import Symbol, simplify
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, EngFormatter
from sympy import pi, sqrt 
from math import log10
#from PyLTSpice import RawRead#LTSpice_RawRead
from sympy.solvers import solve
import sys
#import logging
#from ipywidgets import FloatSlider, interact, interactive
from converter8 import * #works acceptably with converter3

In [3]:
def make_symbols():
    global Rl, Cres, Lres, D, s, Vcmax, Vin, VL, iin, iavg, iC, iR, f_sw, f_res,k, wsw, delta_vC, Tsw
    
    Rl = variable('R_L', 'ohms', None, real=True, positive=True)
    Cres = variable('C_res', 'F', None, real=True, positive=True)
    Lres = variable('L_res', 'H', real=True, positive=True)
    D = variable('D', '%', real=True, positive=True)
    
    f_sw = variable('f_sw', 'Hz', real=True, positive=True)
    f_res = variable('f_res', 'Hz', real=True, positive=True) 

    Vcmax = variable('V_C,max', 'V', real=True)
    Vin = variable('V_g', 'V', real= True)
    VL = variable('V_L', 'V', real= True)

    iin = variable('i_in', 'A', real=True)
    iavg = variable('i_avg', 'A', real=True)
    iC = variable('i_C', 'A', real=True)
    iR = variable('i_R', 'A', real=True)
    wsw = variable(r'\omega_sw', 'A')
    delta_vC = variable(r'\Delta v_C', 'V')
    Tsw = variable('T_sw', real=True)
    k = variable('k', real=True, positive=True)
    return
make_symbols()
#delta_iL

In [4]:
net = converter()
net.add_equation(f_res - 1/(2*pi*sqrt(Lres*Cres))) #VLavg must equal 0 in steady state
net.add_equation(f_res - k*f_sw)
net.add_equation(Cres - iavg/(2*f_res*Vcmax))
net.add_equation(Tsw - 1/f_sw)
net.add_equation(wsw - 2*pi*f_sw)
net.showequations()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [5]:
myprint(f_sw, *net.solver(given={}, want=f_sw))
#myprint(M, *buckboost.solver(given={}, want=M))

<IPython.core.display.Latex object>

In [6]:
print(f' fsw computation test2: {net.computesym(f_sw, {k:3, f_res:1e3})}')
print(f' fsw computation test2: {net.computesym(f_sw, {Tsw:10})}')

 fsw computation test2: 333.3333333333333
keyerror
 fsw computation test2: 0.1
